In [211]:
from typing import Tuple, Any

Distance2D = Tuple[int, int]


class Vector2D:
    
    def __init__(
        self: 'Vector2D',
        horizontal: int,
        vertical: int
    ):
        self.horizontal: int = horizontal
        self.vertical: int = vertical
        
    def reduce_to_single_step_vector(self: 'Vector2D') -> 'Vector2D':
        return Vector2D(
            horizontal = int(self.horizontal / abs(self.horizontal) if self.horizontal else 0),
            vertical = int(self.vertical / abs(self.vertical) if self.vertical else 0)
        )

    def __eq__(self: 'Vector2D', other: 'Vector2D') -> bool:
        return (self.horizontal == other.horizontal) and (self.vertical == other.vertical)
    
    def __isub__(self: 'Vector2D', other: 'Vector2D') -> 'Vector2D':
        self.horizontal -= other.horizontal
        self.vertical -= other.vertical
        return self
    
    def __str__(self: 'Vector2D') -> str:
        return f'Vector2D(horizontal={self.horizontal}, vertical={self.vertical})'
    


# class RelativePosition(Vector2D):
    
#     def __init__(
#         self: Vector2D,
#         pos_a,
#         pos_b
#     )
#         self.horizontal = abs(self.horizontal)
#         self.vertical = abs(self.vertical)
#         super().__init__(horizontal, vertical)
        
        
# class Distance(Vector2D):
    
#     def __init__(
#         self: Vector2D,
#         pos_a,
#         pos_b
#     )
#         self.horizontal = abs(self.horizontal)
#         self.vertical = abs(self.vertical)
#         super().__init__(horizontal, vertical)
                 
        
from dataclasses import dataclass

@dataclass
class Position:

    def __init__(
        self: 'Position', 
        start_x: int = 0,
        start_y: int = 0
    ):
        self.x: int = start_x
        self.y: int = start_y
        
    def touching_directly(self: 'Position', other: 'Position') -> bool:
        touching_horizontally = (abs(self.x - other.x) == 1) and (self.y == other.y)
        touching_vertically =  (abs(self.y - other.y) == 1) and (self.x == other.x)
        return touching_horizontally or touching_vertically
    
    def touching_diagonally(self: 'Position', other: 'Position') -> bool:
        return abs(self.x - other.x) == abs(self.y - other.y) == 1
    
    def __sub__(self: 'Position', other: 'Position'):
        if not isinstance(other, type(self)): 
            return NotImplemented
        return Vector2D(self.x - other.x, self.y - other.y)
    
    def __add__(self: 'Position', vec: Vector2D):
        if not isinstance(vec, Vector2D): 
            return NotImplemented
        return self.__class__(
            start_x=self.x+vec.horizontal,
            start_y=self.y+vec.vertical
        )
    
    def __repr__(self: 'Position') -> str:
        return f'Position(x={self.x}, y={self.y})'
     
    def __str__(self: 'Position') -> str:
        return f'Position(x={self.x}, y={self.y})'
    
    def __hash__(self: 'Position') -> Any:
        return (hash(self.x) ^ hash(self.y))
    
    def __eq__(self: 'Position', other: 'Position'):
        if not isinstance(other, type(self)): 
            return NotImplemented
        return (self.x == other.x) and (self.y == other.y)
    
    def __ne__(self: 'Position', other: 'Position'):
        if not isinstance(other, type(self)): 
            return NotImplemented
        return (self.x != other.x) or (self.y != other.y)
    
class Rope:
    
    def __init__(self: 'Rope'):
        self.head = Position(0, 0)
        self.tail = Position(0, 0)
        self.tail_unique_fields: set[Position] = {Position(0, 0)}

    def move_head(self: 'Rope', step: Vector2D):
        self.head += step
        self._adjust_tail()
            
    def _adjust_tail(self: 'Rope'):
        if self.head.touching_directly(self.tail):
            pass
        elif self.head.touching_diagonally(self.tail):
            pass
        else:
            head_tail_distance = self._calculate_head_tail_difference()
            tail_step = head_tail_distance.reduce_to_single_step_vector()
            self.tail += tail_step
            self.tail_unique_fields.add(self.tail)
    
    def _calculate_head_tail_difference(self: 'Rope') -> int:
        return self.head - self.tail


def parse_line(line: str) -> Vector2D:
    direction, steps = line[0], int(line[2])
    match direction:
        case 'L':
            return Vector2D(horizontal=-1*steps, vertical=0)
        case 'R':
            return Vector2D(horizontal=steps, vertical=0)
        case 'U':
            return Vector2D(horizontal=0, vertical=steps)
        case 'D':
            return Vector2D(horizontal=0, vertical=-1*steps)
        
        
rope = Rope()
with open('input.txt') as file: 
    
    while (line := file.readline()) != '':
        change_head_position = parse_line(line)
        while change_head_position != Vector2D(0, 0):
            change_head_one_step = change_head_position.reduce_to_single_step_vector()
            rope.move_head(change_head_one_step)
            change_head_position -= change_head_one_step

In [1]:
!poetry add aoc -q

In [2]:
!pip install aoc


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from aoc import day

input = [(line.split()[0], int(line.split()[1])) for line in day(9).splitlines()]


def get_vector_change(direction: str):
    match direction:
        case 'L':
            return [-1, 0]
        case 'U':
            return [0, 1]
        case 'R':
            return [1, 0]
        case 'D':
            return [0, -1]
        case _:
            print("Invalid direction argument")
            exit(-1)


def move_tail(head: list[int], tail: list[int]) -> None:
    Δ = [x - y for x, y in zip(head, tail)]

    if abs(Δ[0]) > 1 or abs(Δ[1]) > 1:
        tail[:] = [n + (1 if Δn >= 1 else -1 if Δn <= -1 else 0) for n, Δn in zip(tail, Δ)]


head = [0, 0]
tail = [0, 0]

tail_parts = [[0, 0] for _ in range(9)]

visited_p1 = set()
visited_p2 = set()

for direction, amount in input:
    for i in range(amount):
        head = [x + y for x, y in zip(head, get_vector_change(direction))]

        move_tail(head, tail)
        visited_p1.add(tuple(tail))

        for i in range(len(tail_parts)):
            move_tail(head if i == 0 else tail_parts[i - 1], tail_parts[i])

            if i == 8:
                visited_p2.add(tuple(tail_parts[i]))

print(f"Part 1: {len(visited_p1)}")
print(f"Part 2: {len(visited_p2)}")


ImportError: cannot import name 'day' from 'aoc' (/Users/plis003/Library/Caches/pypoetry/virtualenvs/day-9-r26j2Rq0-py3.10/lib/python3.10/site-packages/aoc/__init__.py)

In [213]:
len(rope.tail_unique_fields)

3045

In [200]:
rope.tail_unique_fields.pop() == rope.tail_unique_fields.pop()

False

In [193]:
part1()

TypeError: part1() missing 1 required positional argument: 'inp'